# Driver for Total Dissolved Gas Prediction and Optimization

## Introduction 

Management and operation of dams within the Columbia River Basin (CRB) provides the region with irrigation, hydropower production, flood control, navigation, and fish passage. These various systemwide demands can require unique dam operations that may result in both voluntary and involuntary spill, thereby increasing tailrace levels of total dissolved gas (TDG) which can be fatal to fish. Appropriately managing TDG levels within the context of the systematic demands requires a predictive framework robust enough to capture the operationally related effects on TDG levels. 

The physical parameters such as spill and hydropower flow proportions, accompanied by the characteristics of the dam such as plant head levels and tailrace depths, are used to develop the empirically-based prediction model.

### Required Technology

This model was built using:
- Python 3 (major packages):
    - Papermill
    - Scipy
    - Numpy
    - Pandas
    - Yaml
    - scikit-learn
    
- Nteract Jupyter Notebooks
    - https://nteract.io/


The majority of the required packages are standard science/engineering python packages when using the Anaconda environment.  Papermill and Jupyter Nteract are a new technology.  Nteract allows Jupyter Notebooks to be parameterized and papermill allows you to access data saved within a single or group of executed notebooks to pass on to other notebooks or scripts. 

## Model Organization

This model consists of 3 parts
1. Configuration file
2. Script file
3. Jupyter Notebooks (including this one)

### Configuration File

The configuration file is in yaml format that consists of the required project pathnames for the Coefficient optimization.  An example configuration file is seen below:


the_dalles:<br>
  &emsp;h_t: TDA.Elev-Tailwater.Ave.1Hour.1Hour.GDACS-COMPUTED-REV<br>
  &emsp;p_atm: TDA.Pres-Air.Inst.1Hour.0.GOES-REV<br>
  &emsp;q_p: TDA.Flow-Gen.Ave.1Hour.1Hour.CBT-REV<br>
  &emsp;q_s: TDA.Flow-Spill.Ave.1Hour.1Hour.CBT-REV<br>
  &emsp;temp_water: TDA.Temp-Water.Inst.1Hour.0.GOES-REV<br>
  &emsp;tdg_f: TDA.%-Saturation-TDG.Inst.1Hour.0.GOES-COMPUTED-REV<br>
  &emsp;tdg_tw: TDDO.%-Saturation-TDG.Inst.1Hour.0.GOES-COMPUTED-REV<br>


### Script File

The script util.py contains the 4 required equations (functions) found in the study "Total dissolved gas prediction and optimization in RIVERWARE," as well as two additional functions for unit conversion.

### Jupyter Notebooks and Model Flow

<img src="../images/driver.png">



The Image above walks through the linear model process and represents each Jupyter Notebook, with the arrow representing this Notebook as the driver of the process.

## Notebooks (let's drive!)

We will start with a parameterized cell for this notebook setting the config file and the project to run through the current optimization process.

In [ ]:
config_file = '../config.yml'
project = 'the_dalles'
lookback = 365*3
DATA_DIR = '../projects/{}'
maxiter = 2
bootstraps = 5

In [ ]:
data_dir = DATA_DIR.format(project)

In [ ]:
#creating directories to store model results

import os

directories = ['/error', '/results', '/train_test']
for directory in directories:
    os.mkdir(data_dir+directory) 



### Get Data

Now let's get the data from the CWMS database.  I use a pre-canned function `get_cwms` that I created to extract data from CWMS and return a pandas dataframe.  This will be the 'raw' data for the project.  I say 'raw' because there is a process to interpolate that limits itself to 5 missing values.  Furthermore the data is converted into SI units because that is 



#### Parameters

 - project_dict
 - lookback
 
#### Notebook

[get_data.ipynb](get_data.ipynb)


In [ ]:
import yaml
import papermill as pm
config = yaml.load(open(config_file))
project_dict = config[project]
saved_notebook = data_dir + '/{}_raw.ipynb'.format(project)

pm.execute_notebook(
   'get_data.ipynb',
   saved_notebook,
   parameters = dict(project = project, project_dict=project_dict, lookback=lookback)
)



In [ ]:
#get raw data to pass to split notebook
nb = pm.read_notebook(saved_notebook)
nb_df = nb.dataframe
raw_data = nb_df[nb_df['name']=='raw_data']['value'].values[0]


### Split → Save Data
I will now use two notebooks jointly, `split_data.ipynb` and `save_data.ipynb`.  This model will use a k-fold cross-validation with data split by year.  Each year will be a test dataset one time with the remaining years as the train set.  The hourly time series data is highly correlated in time so a random split would not produce an appropriate validation.  TDG offers a cyclic pattern where each year can be assumed independent of any other.  This will produce much better generalizations than a random split.  The split notebook feeds directly into the save notebook. So the data will be split and then saved in individual notebooks labeled by year that is the test set.


#### Parameters
 - project 
 - frequency

#### Notebooks
 [split_data.ipynb](split_data.ipynb)
 
 [save_data.ipynb](save_data.ipynb)

In [ ]:
#parameters
raw_data = raw_data
freq = 'Y' 
data_dir = data_dir
project = project

pm.execute_notebook(
   'split_data.ipynb',
   data_dir + '/{}_grouped.ipynb'.format(project),
   parameters = dict(raw_data=raw_data, freq=freq, data_dir=data_dir, project=project)
)


### Optimize
This is the heart of the model.  This will notebook optimizes the three coefficients *b1, b2, b3* by minimizing the root mean squared error.  The results of this model will feed into a monte carlo simulation of another model.  Therefore, I will create around 1000 models from the available data in a bootstrap process with each year providing 1000/n years models.  The distribution of coefficients will be used to construct the monte carlo model.  

I will use the `minimize` function from [scipy's optimize] (https://docs.scipy.org/doc/scipy/reference/optimize.html) package.  It requires a first guess for the model coefficients.  I will optimize on all the data first as I think this will be a good first guess for each model.  Within year model bootstrap models will use an average of the previous model runs as a first guess.
 

#### Parameters
 - data
 - x0
 - sample
 - maxiter

The x0 parameter is the initial guess to start the optimization algorithm with.  The sample parameter is a boolean of whether to sample the data with replacement used for the bootstrap process.  The maxiter is the maximum number of iterations for the optimization algorithm before it breaks.


#### Notebook
[optimize.ipynb](optimize.ipynb)
 

In [ ]:
#parameters 
data = raw_data
sample = False
maxiter = maxiter
x0 = [1,.5,150]

optimized = data_dir + '/optimized.ipynb'

pm.execute_notebook(
           'optimize.ipynb',
           optimized,
           parameters = dict(data=raw_data, sample=False, maxiter=maxiter, x0=x0)
        )


In [ ]:
#get raw data to pass to split notebook and x0 to pass to error
nb = pm.read_notebook(optimized)
nb_df = nb.dataframe
x = nb_df[nb_df['name']=='x']['value'].values[0]
x

### Error

I will now calculate the error of the model by testing against the training data.

#### Parameters
    - b
    - test_data
#### Notebook
[error.ipynb](error.ipynb)

In [ ]:
#parameters
test_data = raw_data
b = x
error = data_dir + '/errors.ipynb'

pm.execute_notebook(
   'error.ipynb',
   error,
   parameters = dict(test_data = test_data, b = b,)
   )



### Validation

I want to create a graphical representation on how well the model fits the actual data

#### Parameters
    - error_directory
    - error_notebook
#### Notebook
[validation.ipynb](validation.ipynb)

In [ ]:
#parameters
error_notebook = data_dir + '/errors.ipynb'
validation = data_dir + '/validation.ipynb'
pm.execute_notebook(
   'validation.ipynb',
   validation,
   parameters = dict(error_notebook = error_notebook)
   )

## Bootstrap

I will now run through the process again in a bootstrap to obtain a distribution of b coefficient values.  The bootstrap notebook runs through the optimization and error notebooks for each year.  A year is set as the test data and the otehr years are set as the train data.  Samples of the train data are optimized bootstraps/n year times to create a distribution of b coefficients to sample from in a future monte carlo model.

### Parameters
    - data_files
    - maxiter
    - bootstraps
    - data_dir
    - b



In [ ]:
import glob

data_files = glob.glob(data_dir+'/train_test/*.ipynb')
bootstrap_output = data_dir +'/bootstrap.ipynb'
pm.execute_notebook(
   'bootstrap.ipynb',
   bootstrap_output,
   parameters = dict(data_files=data_files, data_dir=data_dir, maxiter=maxiter, bootstraps=bootstraps, b=b)
   )




## Validation
Run The validation notebook on all of the bootstrapped error notebooks

In [ ]:
error_directory = data_dir + '/error'
bootstrap_validation = data_dir + '/bootstrap_validation.ipynb'
pm.execute_notebook(
   'validation.ipynb',
   bootstrap_validation,
   parameters = dict(error_directory = error_directory)
   )
